In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

### Working with results of a ZenGarden run

This notebook shows some examples on how to handle the results of a `zen-garden` run. It uses the outputs of the tests as examples. Therefore, you need to run the tests via 
```
coverage run --source="zen_garden" -m pytest -v tests/testcases/run_test.py
```
to get the data.

### Reading in the results

The results can be read in using the `Results` class. This will automatically read in all relevant files, independent of the number of scenarios or if the optimization was run with perfect or myopic foresight. However, you should know what type of input you are reading in, as the output of some routines will depend on the type.

In [6]:
# This has no scenarios and perfect foresight
res_basic = Results("../outputs/test_1a/")
# Here we have three different scenarios
res_scenario = Results("../outputs/test_6a/")
# This was run with myopic foresight
res_mf = Results("../outputs/test_4g/")
# This was run with aggregated time steps and multi year
res_ts = Results("../outputs/test_4f/")

/Users/Ludwig/Library/CloudStorage/OneDrive-ETHZürich/Master_Thesis/ZEN-garden/zen_garden/model/objects/time_steps.py:175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  time_steps_combi = np.vstack(pd.unique(list(zip(sequence_operation, sequence_yearly)))).T


FileNotFoundError: The file does not exists as json or gzip: /Users/Ludwig/Library/CloudStorage/OneDrive-ETHZürich/Master_Thesis/ZEN-garden/outputs/test_6a/scenario_/system

### Accessing the config

The standard dictionaries, i.e. system, analysis, scenarios, solver and dict_sequence_time_steps can be accessed via the results dict.

In [ ]:
# Show the analysis settings
pprint(res_basic.results["analysis"])

### Comparing the configs, parameters and variables of two result files

You can compare two result objects by using the `compare_configs`, `compare_model_parameters` and `compare_model_variables` classmethods. This can help you to get a fast overview of where two datasets are different and where the reasons for errors lie.

In [ ]:
# difference between configs
diff_config = Results.compare_configs([res_basic,res_ts])
diff_config

In [ ]:
# difference between model params
# for larger models, this can take several minutes
diff_params = Results.compare_model_parameters([res_ts,res_mf])
diff_params

In [ ]:
# you can also compare the model variable values. However, even a small change in the input data might have a significant impact on the results, which then leads to large differences in the variable values
diff_var = Results.compare_model_variables([res_scenario,res_scenario],scenarios=("scenario_2","scenario_4"))
diff_var

### Extracting individual data frames
There are three main methods to extract component (parameter and variable) values: `get_total`, `get_full_ts`, and `get_df`. It does not matter if the component is a parameter or a variable.

You can get the total value of a component for each year by using `get_total`. The full time series of a component is returned by `get_full_ts`. This reverts the time series aggregation and disaggregates the data. If the selected component is indexed by the years (e.g., `capex` or `capacity`), then the returned data is the same as from `get_total`. If you specify a `year`, only the values for this year are returned. You can select a specific `element_name`, which corresponds to the first level of the index, i.e., generally a technology or a carrier.

You can either feed `get_total` and `get_full_ts` a dataframe as returned by `get_df` or specify the component name as a string

You can access individual data frames from any component with the `get_df` method. However, the behavior differs slightly for the usecases.
If the optimization was run with perfect foresight and no scenarios, then the data frame is returned as `pandas.Series`. For myopic foresight, the final dataframe is stiched together from the relevant outputs if the component is set yearly. If multiple scenarios were run, a dictionary containing the data frame for all different scenarios is returned. However, you can specify the scenario in advance, then only a single data frame is returned.

Additionally, the method provides a shortcut to save the extracted dataframe directly to a CSV, or in case of different scenarios to a bundle of CSV files.

In [ ]:
# This will return a single data frame
df = res_ts.get_df('flow_import')
df

In [ ]:
# We can calculate the total value of the dataframe
res_ts.get_total(df)

In [ ]:
# We can also only specify the name
res_ts.get_total("flow_import")

In [ ]:
# and also get the full timeseries if we want
df_full_ts = res_ts.get_full_ts("flow_import")
df_full_ts

In [ ]:
# you can also save the dataframe directly to a csv:
df = res_basic.get_df('carbon_emissions_overshoot', to_csv="test")
# this prints out the content of the generated csv
!cat test.csv

In [ ]:
# if we have multiple scenarios, all dataframe will be returned in a dict
df_dict = res_scenario.get_df('flow_export')
pprint(df_dict)

In [ ]:
# saving this to a csv will create a file for each scenario
df_dict = res_scenario.get_df('export_flow', to_csv="test2")
# list files and print content of one
!echo "Generated files: "
!ls test2*
!echo
!echo "Output of test2_scenario_2.csv"
!cat test2_scenario_2.csv

In [ ]:
# we can also specify a specific scenario
df = res_scenario.get_df('carrier_flow', scenario="scenario_2")
df

In [ ]:
# This will collect the values from the results folders of the different myopic foresight runs
df = res_mf.get_df("carbon_emissions_cumulative")
df

If you enable the calculation of dual variables (e.g, the marginal cost of production of a carrier as the shadow price of the energy balance) in the config with `solver["add_duals"] = True`, you can use `get_dual` to extract the dual variables. Otherwise, it is skipped. `get_dual` uses `get_full_ts`, thus always returns the full disaggregated time series. You must pass the name of the constraint.

In [ ]:
res_mf.get_dual("constraint_nodal_energy_balance")

### How to plot your results

The ```Results``` class offers three different plotting functions explained at https://github.com/RRE-ETH/ZEN-garden/discussions/251

In [ ]:
#Create the standard plots of your Results object (e.g. res_ts)
res_ts.standard_plots()

In [ ]:
#Visualise the energy balance at the node Germany for the carrier heat in year 0
res_ts.plot_energy_balance("DE","heat",0)
#create a zoom-in
res_ts.plot_energy_balance("DE", "heat", 0, start_hour=int(8760/2), duration=5*24)
#save your figure as a pdf(works identically for all three plot functions)(figure is shown anyway)
res_ts.plot_energy_balance("DE", "heat", 0, start_hour=int(8760/2), duration=5*24, save_fig=True)
#save your figure in another format
res_ts.plot_energy_balance("DE", "heat", 0, start_hour=int(8760/2), duration=5*24, save_fig=True, file_type="png")

In [ ]:
#Create more customized plots:
res_ts.plot("input_flow")
#time adjustments
#plot only second year
res_ts.plot("input_flow", year=1)
#create zoom-in
res_ts.plot("input_flow", year=1, start_hour=0, duration=500)
#use yearly time steps
res_ts.plot("input_flow", yearly=True)

In [ ]:
res_ts.plot("capacity")
#data adjustments
#plot specific technology type
res_ts.plot("capacity", tech_type="storage")
#plot charging/discharging capacities
res_ts.plot("capacity", tech_type="storage_power")
#plot all nodes separately
res_ts.plot("capacity", tech_type="conversion", node_edit="all")
#plot data of a single node
res_ts.plot("capacity", node_edit="CH")

In [ ]:
res_ts.plot("output_flow")
#sum technologies of identical output carrier (effect can't really be seen as there is only one tech in the dataset)
res_ts.plot("output_flow", sum_techs=True)
#extract technologies of specific reference carrier (effect can't really be seen as there is only one tech in the dataset)
res_ts.plot("output_flow", reference_carrier="heat")

In [ ]:
#create plot of multi-scenario dataset (standard scenario is plotted)
res_scenario.plot("input_flow")
#specify another scenario
res_scenario.plot("input_flow", scenario="scenario_4")